# Nous souhaitons modéliser un ensemble de données structurés. Un "data frame" est structuré avec les individus en lignes et les différentes variables en colonne.
équivalent à une table de BD, ou une dataTable.

Par exemple:

In [6]:
champs=['Nom', 'Prenom', 'tel', 'email']
lignes = [['Pascal', 'Fares', '03xxxyyy', 'pf@isae.edu.lb'], 
          ['Emile', 'Dupont', '336xxyyzzww', 'de@isae.edu.lb']]

Représenterait:

| Nom | Prenom | Tel | Email |
| --- | ------ | --- | ----- |
|Pascal|Fares|03xxxyyy|pf@isae.edu.lb|
|Emile|Dupont|336xxyyzzww|de@isae.edu.lb|

## Pour récuperer l'index d'un champs

In [8]:
# La fonction getIndex,si le champs pas dans la liste => exception valueError
getIndex = lambda listeChamps, nomChamp: listeChamps.index(nomChamp)

In [9]:
getIndex(['Nom', 'Prenom', 'tel', 'email'], 'email')

3

In [10]:
['Pascal', 'Fares', '03xxxyyy', 'pf@isae.edu.lb'][getIndex(['Nom', 'Prenom', 'tel', 'email'], 'tel')]

'03xxxyyy'

## Quand on a l'index on pourrait récupérer la valeur des champs
En utilisant map, ou list comprehetion

In [12]:
# Liste comprenhention
[ligne[getIndex(champs,'email')] for ligne in lignes]

['pf@isae.edu.lb', 'de@isae.edu.lb']

In [13]:
#map
list(map(lambda ligne: ligne[getIndex(champs, 'email')], lignes))

['pf@isae.edu.lb', 'de@isae.edu.lb']

## Réalisons la fonction : l'équivalent de select en SQL (la projection): en fonctionnel par un  `map`

### Paramètres nécessaires: tous les champs : `champs`, la selection de champs : `schamps`, les donnees : `donnees`

* **propriétés:** schamps sous ensemble de champs
* **resultat attendu:** la listes des ensembles de valeurs correspondants au `schamps` issues des `donnees`

In [2]:
# Fonctions liste schamps -> liste indexe
tousLesIndexes = lambda champs: lambda schamps: [champs.index(c) for c in schamps]

In [3]:
tli = lambda champs,schamps : [champs.index(c) for c in schamps]

In [4]:
#ceci va permettre de définir par exemple
indexPourMesChamps = tousLesIndexes(['Nom', 'Prenom', 'tel', 'email'])
# et donc réaliser ceci
indexPourMesChamps(['Prenom', 'email'])

[1, 3]

In [5]:
tousLesIndexes(['Nom', 'Prenom', 'tel', 'email'])(['Prenom', 'email'])

[1, 3]

In [22]:
#Fonction qui a partir d'un eliste d'indexes renvoie la liste de valeurs
getValeurs = lambda l,indexes: [l[i] for i in indexes]

# Exemples:
print(getValeurs(['Pascal', 'Fares', '03xxxyyy', 'pf@isae.edu.lb'],[1, 3]))
print(getValeurs(['Pascal', 'Fares', '03xxxyyy', 'pf@isae.edu.lb'],tousLesIndexes(['Nom', 'Prenom', 'tel', 'email'])(['Prenom', 'email'])))

['Fares', 'pf@isae.edu.lb']
['Fares', 'pf@isae.edu.lb']


In [24]:
#maintenant réupérons toutes les valeurs
select = lambda champs, donnees: lambda schamps: [getValeurs(d, tousLesIndexes(champs)(schamps)) for d in donnees]

In [30]:
print(f"Champs = {champs} \nlignes = {lignes}")
select(champs,lignes)(['Prenom', 'email'])

Champs = ['Nom', 'Prenom', 'tel', 'email'] 
lignes = [['Pascal', 'Fares', '03xxxyyy', 'pf@isae.edu.lb'], ['Emile', 'Dupont', '336xxyyzzww', 'de@isae.edu.lb']]


[['Fares', 'pf@isae.edu.lb'], ['Dupont', 'de@isae.edu.lb']]

In [31]:
#Ou bien
maBD=select(['Nom', 'Prenom', 'tel', 'email'],
            [['Pascal', 'Fares', '03xxxyyy', 'pf@isae.edu.lb'], 
             ['Emile', 'Dupont', '336xxyyzzww', 'de@isae.edu.lb']
            ])
print(maBD(['Prenom', 'email']))
print(maBD(['Nom', 'tel']))

[['Fares', 'pf@isae.edu.lb'], ['Dupont', 'de@isae.edu.lb']]
[['Pascal', '03xxxyyy'], ['Emile', '336xxyyzzww']]


In [33]:
# Un autre exemple de DataFrame idEleve, edmatiere, note
maBDMatiereNotes=select(['idMat','idEtud','note'],
                        [[503, 1001, 11],
                         [503, 1002, 12],
                         [503, 1003, 9],
                         [504, 1001, 11],
                         [504, 1002, 10],
                        ])
maBDMatiereNotes(['idEtud','note'])

[[1001, 11], [1002, 12], [1003, 9], [1001, 11], [1002, 10]]

> Mais je ns souhaiterais que les notes de l'étudiant 1001, pas les autres
# Le filtrage des données : l'équivalent du where en SQL
cette fois si ce n'est pas un sous ensemble d'attribut que l'on souahite mais les lignes (les donnés) qui correspondent à un certain critères (ou prédicat)

Le prédicat que nous souhaitons est un prédicat qui s'applique à un champs, par exemple:
* idEtud == 1001
* note > 10
* ou toute autre combinaisons

## commencons par définir un prédicat associé a un champ


In [54]:
#Soit la fonctions where : un filter (ou comprenhemtion list)
# Les donnees, liste des champs, un champs, un predicat => renvoie True ou False en fonction du résultat de l'application du prédicat
where=lambda lchamps,donnees: lambda champ,p: [l for l in donnees if p(l[getIndex(lchamps,champ)])]
#ou

In [55]:
WhereMatiereNotes=where(['idMat','idEtud','note'],
                        [[503, 1001, 11],
                         [503, 1002, 12],
                         [503, 1003, 9],
                         [504, 1001, 11],
                         [504, 1002, 10],
                        ])
mnChamps=['idMat','idEtud','note']
mnDonnees=[[503, 1001, 11],
           [503, 1002, 12],
           [503, 1003, 9],
           [504, 1001, 11],
           [504, 1002, 10]
          ]

In [56]:
WhereMatiereNotes('idEtud', lambda ie: ie==1001)

[[503, 1001, 11], [504, 1001, 11]]

In [ ]:
### Avec filter au lieux de list comprehention (en découpant les étapes)

In [57]:
idetudP=lambda ie: ie==1001 #Le prédicat que nous souhaitons appliquer


In [58]:
filter(lambda l: idetudP(l[getIndex(mnChamps, 'idEtud')]), mnDonnees )

In [59]:
list(filter(lambda l: idetudP(l[getIndex(mnChamps, 'idEtud')]), mnDonnees ))

[[503, 1001, 11], [504, 1001, 11]]

# Tout ceci est un peu verbeux (bavard) et éparpillé, passons au paradigme Objet, tout en gradant un esprit fonctionel
## Quelques remarques

* Nous avons définit des fonctions qui ont tout ce qui faut en paramétre pour réliser leur objectif, et sans avoir besoin d'information sauvegardée ailleurs. Nous avons essayer de faire du fonctionel pure.
* Nous aurions pu définir des fonctions qui récupère certains données depuis une source de donnée (une mémoire, un disque, un réseux, ...) [Par exemple ici](#55)
* Nous pourrions nous aboné à un flux de données, et réagir a chauqe nouvelle donnée recut, c'est le paradigme réactif